<a href="https://colab.research.google.com/github/Swarup2608/Fake_news_detection/blob/main/fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Set Contents**


1.   id: Id number of the Data Article
2.   title: Title of the news Article
3.   author: Author of the news Article
4.   text: Text of the news Article; Which can be incomplete in many cases
5. label: a label that marks whether the news article is real or fake

# **Label is 1 or 0 **
**1: Fake    0: Real**



In [68]:
# Dependenices for the detection
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [69]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [70]:
# Reading Data Set
df = pd.read_csv('train.csv')

In [71]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [72]:
df.shape

(20800, 5)

In [73]:
# Searching for missing values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [74]:
df = df.fillna('')

In [75]:
# merging the author name and news title
df['content'] = df['author']+' '+df['title']

In [76]:
df['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [77]:
# separating the data & label
X = df.drop(columns='label', axis=1)
Y = df['label']

In [78]:
# Adding stemmer to stem the words programs -> program
stemmer = PorterStemmer()

In [79]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [80]:
df['content'] = df['content'].apply(stemming)

In [81]:
#separating the data and label of stemmed data
X = df['content'].values
Y = df['label'].values

In [82]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [83]:
# Splitting data to training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [84]:
# Training the model of Logistic Regression
model = LogisticRegression()

In [85]:
model.fit(X_train,Y_train)

LogisticRegression()

In [86]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9865985576923076


In [87]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


In [88]:
X = pd.read_csv('test.csv')
X

,id,title,author,text,label
0,1,Donald Trump Sends Out Embarrassing New Year’s...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,NaN


In [89]:
X['label'] = 1
# merging the author name and news title
X['content'] = X['author']+' '+X['title']

In [90]:
# separating the data & label
Xn = X.drop(columns='label', axis=1)
Yn = X['label']

In [91]:
X['content'] = X['content'].apply(stemming)
#separating the data and label of stemmed data
Xn = X['content'].values
Yn = X['label'].values
X = vectorizer.transform(Xn)

In [92]:
prediction = model.predict(X)
if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

The news is Fake
